In [6]:
# List of library and resources needed
import pandas as pd
import nltk 
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
import string
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import joblib

#nltk.download('stopwords')
#nltk.download('punkt')

In [13]:
stop_words = set(stopwords.words('english'))

# Function to clean sentences using regex
def clean_sentence(sentence):
    # Convert to lowercase
    sentence = sentence.lower()
    # Remove non-alphanumeric characters using regex
    sentence = re.sub(r'[^a-zA-Z0-9\s]', '', sentence)
    # Tokenize the sentence
    words = word_tokenize(sentence)
    # Remove stopwords
    words = [word for word in words if word not in stop_words]
    return ' '.join(words)

In [14]:
# Load the saved CSV file
loaded_combined_data = pd.read_csv('combined_data_with_labels.csv')

# Extract labels from the loaded data
labels = loaded_combined_data['label']

# Split the data into features (sentences) and labels
X = loaded_combined_data['sentence']
y = loaded_combined_data['label']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=93)

# Initialize the TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit the vectorizer on the training sentences and transform them into TF-IDF vectors
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)

# Transform the test sentences into TF-IDF vectors
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Initialize the logistic regression model
logistic_regression_model = LogisticRegression()

# Train the model on the training data
logistic_regression_model.fit(X_train_tfidf, y_train)

# Predict labels for the test data
y_pred = logistic_regression_model.predict(X_test_tfidf)

# Calculate accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.9672131147540983


In [15]:
# Generate classification report
report = classification_report(y_test, y_pred)

# Print the classification report
print(report)


              precision    recall  f1-score   support

           0       1.00      0.87      0.93        30
           1       0.95      0.95      0.95        43
           2       0.96      1.00      0.98       110

    accuracy                           0.97       183
   macro avg       0.97      0.94      0.95       183
weighted avg       0.97      0.97      0.97       183



In [16]:
# Save the trained model to a file
joblib.dump(logistic_regression_model, 'gt_dt_et_regression_model.pkl')

['gt_dt_et_regression_model.pkl']

In [18]:
# Load the saved model
loaded_model = joblib.load('gt_dt_et_regression_model.pkl')

# Define a function to preprocess a single sentence
def preprocess_sentence(sentence):
    # Clean the sentence
    cleaned_sentence = clean_sentence(sentence)
    # Transform the cleaned sentence into TF-IDF vector
    tfidf_vector = tfidf_vectorizer.transform([cleaned_sentence])
    # Convert TF-IDF vector to a dense representation (array)
    tfidf_vector_array = tfidf_vector.toarray()
    return tfidf_vector_array

# Define a function to predict labels for a single sentence
def predict_label(sentence):
    # Preprocess the sentence
    preprocessed_sentence = preprocess_sentence(sentence)
    # Predict label using the loaded model
    prediction = loaded_model.predict(preprocessed_sentence)
    return prediction[0]

# Get user input for the sentence to be tested
user_sentence = input("Enter a sentence to be tested: ")

# Predict the label for the user input sentence
predicted_label = predict_label(user_sentence)

# Map predicted label to corresponding category
category = ""
if predicted_label == 1:
    category = "Displacement"
elif predicted_label == 0:
    category = "Energy"
else:
    category = "Not displacement or energy"

print("Predicted Category:", category)

Enter a sentence to be tested: the man is full of kinetic energy
Predicted Category: Energy
